In [2]:
import pandas as pd 
import numpy as np
import time
import requests

### Historical data from the odds api, but i have to pay. Worst case ill pay, if im not able to scrap from betting pros

In [ ]:
import requests
base_url = "https://api.the-odds-api.com/v4/sports/basketball_nba/events/"
API_KEY = '8aa8bb1fc5d56c98b45d6f3f58beab70'
Sport = 'basketball_nba'
Regions = 'us'
Market = 'h2h,odds'
Odds_format = 'american'
Date_format = 'iso' 
date = '2021-10-22T22:45:00Z'
id = '34e829d65b6a0cbb49573338a86772ec'

url = (
    f"https://api.the-odds-api.com/v4/historical/sports/{Sport}/events"
    f"?apiKey={API_KEY}"
    f"&date={date}"
    f"&regions={Regions}"
)

response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to get events: {response.status_code}, {response.text}")
else:
    events = response.json()
    for event in events:
        print(event['id'], event['commence_time'], event['home_team'], event['away_team'])

### Grabs players odds for the day(US all boookmakers, DFS is prizepicks and underdogs)

In [2]:
from NBAPropFinder.NBAPropFinder import NBAPropFinder

nba_props = NBAPropFinder()
prizePicks = nba_props.dataframe
prizePicks.to_csv('PROPS_DATA/Playoffs_DFS(6_11_25).csv')

Scraping Odds API...
Organizing Data...


### Single Bets from bookmakers that dont include prizePicks or UnderDogs

In [3]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
all_bookmakers = pd.read_csv('./PROPS_DATA/Playoffs_DFS(6_11_25).csv')
prop_type = all_bookmakers['CATEGORY'].unique()
print(prop_type)
all_bookmakers

['player_points' 'player_rebounds' 'player_assists' 'player_threes'
 'player_blocks' 'player_steals' 'player_rebounds_assists'
 'player_blocks_steals']


,Unnamed: 0,BOOKMAKER,CATEGORY,NAME,OVER/UNDER,LINE,ODDS
0,0,PrizePicks,player_points,Jalen Williams,Over,0.5,-137
1,1,PrizePicks,player_points,Shai Gilgeous-Alexander,Over,33.5,-137
2,2,PrizePicks,player_points,Shai Gilgeous-Alexander,Under,33.5,-137
3,3,PrizePicks,player_points,Tyrese Haliburton,Over,17.5,-137
4,4,PrizePicks,player_points,Tyrese Haliburton,Under,17.5,-137
...,...,...,...,...,...,...,...
411,411,DraftKings Pick6,player_blocks_steals,Cason Wallace,Under,1.5,-137
412,412,DraftKings Pick6,player_blocks_steals,Luguentz Dort,Over,1.5,-137
413,413,DraftKings Pick6,player_blocks_steals,Luguentz Dort,Under,1.5,-137
414,414,DraftKings Pick6,player_blocks_steals,Myles Turner,Over,2.5,-137


In [9]:
all_bookmakers.to_csv('PROPS_EV/singleBets(6_11_25).csv')

In [6]:
import scipy.stats as stats

def single_bet(data,bookmakers,category ='player_points', stat_line = 'PTS'):  
    Props = bookmakers[['NAME', 'BOOKMAKER', 'CATEGORY','LINE','OVER/UNDER', 'ODDS']].loc[bookmakers['CATEGORY'] == category]
    results = []

    for idx, row in Props.iterrows():
        name = row['NAME']
        bookmaker = row['BOOKMAKER']
        category = row['CATEGORY']
        line = row['LINE']
        over_under = row['OVER/UNDER']
        odds = row['ODDS']

        # Filter player data
        df = data.loc[data['PLAYER_NAME'] == name]
        if df.empty:
            continue

        # Mean and std dev of actual results
        mean = round(df[stat_line].mean(), 2)
        std = round(df[stat_line].std(), 2)
        if std == 0:
            continue

        # Z-score-based probability
        z_score = (line - mean) / std
        norm_prob = 1 - stats.norm.cdf(z_score)
        if over_under == "Under":
            norm_prob = 1 - norm_prob

        # Monte Carlo simulation
        sim_points = np.random.normal(mean, std, 10000)
        sim_prob = np.mean(sim_points > line)
        if over_under == "Under":
            sim_prob = 1 - sim_prob

        # Weighted average of sim and norm prob
        final_prob = 0.8 * sim_prob + 0.2 * norm_prob
        
        # EV calculation
        stake = 100
        profit = (odds / 100) * stake if odds > 0 else (100 / abs(odds)) * stake
        payout = stake + profit
        ev = (final_prob * profit) - ((1 - final_prob) * stake)

        # Kelly Criterion
        def kelly_criterion(prob, payout, stake):
            net_profit = payout - stake
            loss_prob = 1 - prob
            kelly = (net_profit * prob - loss_prob * stake) / net_profit
            return max(0, round(kelly, 4))

        kelly = kelly_criterion(final_prob, payout, stake)

        results.append({
            'NAME': name,
            'BOOKMAKER': bookmaker,
            'CATEGORY': category,
            'LINE': line,
            'OVER/UNDER': over_under,
            'ODDS': odds,
            'SIM PROB': round(final_prob, 3),
            'EV': round(ev, 2),
            'KELLY CRITERION': kelly,
        })

    return pd.DataFrame(results).sort_values(by='EV', ascending=False).reset_index(drop=True)

current = single_bet(data, all_bookmakers, category='player_points', stat_line='PTS')
current.head(15)
# current.to_csv('PROPS_EV/singleBets(6_9_25).csv')


,NAME,BOOKMAKER,CATEGORY,LINE,OVER/UNDER,ODDS,SIM PROB,EV,KELLY CRITERION
0,Isaiah Hartenstein,BetRivers,player_points,6.5,Over,-110,0.731,39.65,0.4361
1,Isaiah Hartenstein,FanDuel,player_points,6.5,Over,-112,0.733,38.81,0.4347
2,T.J. McConnell,Bovada,player_points,8.5,Over,170,0.505,36.39,0.2140
3,Isaiah Hartenstein,BetOnline.ag,player_points,6.5,Over,-119,0.736,35.37,0.4209
4,Isaiah Hartenstein,Bovada,player_points,8.5,Over,150,0.540,35.11,0.2341
5,Isaiah Hartenstein,Bovada,player_points,7.5,Over,110,0.640,34.33,0.3121
6,Isaiah Hartenstein,DraftKings,player_points,6.5,Over,-120,0.732,34.21,0.4105
7,T.J. McConnell,Bovada,player_points,6.5,Over,100,0.655,31.09,0.3109
8,Isaiah Hartenstein,Bovada,player_points,9.5,Over,205,0.427,30.30,0.1478
9,Isaiah Hartenstein,Bovada,player_points,6.5,Over,-130,0.728,28.78,0.3742


## PrizePicks & Underdog
## 2 leg EVs since we cant do single bets on those platforms

In [1]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
pp_data = pd.read_csv('PROPS_DATA/Playoffs_DFS(6_11_25).csv')
prizePicks = pp_data[['NAME', 'BOOKMAKER', 'CATEGORY','LINE','OVER/UNDER', 'ODDS']]
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    'player_blocks': 'BLK',
    'player_steals': 'STL',
    'player_turnovers': 'TOV',
}

NameError: name 'pd' is not defined

### Functions used to grab the ev and kelly

In [21]:
from scipy.stats import norm

def zscore_prob(mean, std, line, side="over"):
    z = (line - mean) / std
    prob = 1 - norm.cdf(z)
    return prob if side == "over" else 1 - prob

def kelly_criterion(probability, payout, stake, kelly_fraction=1.0):
    netProfit = payout - stake
    probabilityOfLoss = 1 - probability
    kelly = (netProfit * probability - probabilityOfLoss) / netProfit
    return max(0, round(kelly * kelly_fraction, 4))

def get_combined_stat_values(data, player_name, stat_line):
    """Helper function to get stat values for single or combined stats"""
    df = data.loc[data['PLAYER_NAME'] == player_name]
    
    if df.empty:
        return pd.Series()
    
    if '+' in stat_line:
        # Handle combined stats
        stat_parts = stat_line.split('+')
        return df[stat_parts].sum(axis=1)
    else:
        # Single stat
        return df[stat_line]

def prizePicksPairsEV(data, prizePicks, propDict, simulations=10000, stake=100, payout=300, weight=0.8):
    def bootstrapSim(name1, name2, data, prizePicks1, prizePicks2, simulations, stake, payout, weight, stat_line1, stat_line2):
        df1 = data.loc[data['PLAYER_NAME'] == name1]
        df2 = data.loc[data['PLAYER_NAME'] == name2]

        if df1.empty or df2.empty:
            return None

        line1 = prizePicks1.loc[prizePicks1['NAME'] == name1, 'LINE'].values[0]
        line2 = prizePicks2.loc[prizePicks2['NAME'] == name2, 'LINE'].values[0]

        # Get stat values for bootstrap
        stat1_values = get_combined_stat_values(data, name1, stat_line1)
        stat2_values = get_combined_stat_values(data, name2, stat_line2)
        
        if len(stat1_values) == 0 or len(stat2_values) == 0:
            return None

        # Bootstrap simulation - sample from actual historical data
        bootstrap1 = np.random.choice(stat1_values, size=simulations, replace=True)
        bootstrap2 = np.random.choice(stat2_values, size=simulations, replace=True)

        over1, under1 = bootstrap1 > line1, bootstrap1 <= line1
        over2, under2 = bootstrap2 > line2, bootstrap2 <= line2

        bootstrap_probs = {
            'OVER/OVER': np.mean(over1 & over2),
            'UNDER/UNDER': np.mean(under1 & under2),
            'OVER/UNDER': np.mean(over1 & under2),
            'UNDER/OVER': np.mean(under1 & over2),
        }

        # Calculate z-score probabilities for comparison/weighting
        mean1, std1 = stat1_values.mean(), stat1_values.std()
        mean2, std2 = stat2_values.mean(), stat2_values.std()
        
        if std1 == 0 or std2 == 0:
            # If no variance, use pure bootstrap
            final_probs = bootstrap_probs
        else:
            z_over1 = zscore_prob(mean1, std1, line1, side='over')
            z_under1 = 1 - z_over1
            z_over2 = zscore_prob(mean2, std2, line2, side='over')
            z_under2 = 1 - z_over2

            z_probs = {
                'OVER/OVER': z_over1 * z_over2,
                'UNDER/UNDER': z_under1 * z_under2,
                'OVER/UNDER': z_over1 * z_under2,
                'UNDER/OVER': z_under1 * z_over2,
            }

            # Weighted combination of bootstrap and z-score
            final_probs = {
                k: weight * bootstrap_probs[k] + (1 - weight) * z_probs[k]
                for k in bootstrap_probs
            }

        evs = {k: round((final_probs[k] * payout) - stake, 2) for k in final_probs}

        return {
            'OVER/OVER': {'prob': round(final_probs['OVER/OVER'], 4), 'ev': evs['OVER/OVER']},
            'UNDER/UNDER': {'prob': round(final_probs['UNDER/UNDER'], 4), 'ev': evs['UNDER/UNDER']},
            'OVER/UNDER': {'prob': round(final_probs['OVER/UNDER'], 4), 'ev': evs['OVER/UNDER']},
            'UNDER/OVER': {'prob': round(final_probs['UNDER/OVER'], 4), 'ev': evs['UNDER/OVER']}
        }

    all_pairs = []
    seen = set()

    # Iterate through all categories for player 1
    for cat1 in propDict:
        prizePicks1 = prizePicks[prizePicks['CATEGORY'] == cat1]
        stat_line1 = propDict[cat1]
        
        # Compare against all categories (including same category)
        for cat2 in propDict:
            prizePicks2 = prizePicks[prizePicks['CATEGORY'] == cat2]
            stat_line2 = propDict[cat2]
            
            for name1 in prizePicks1['NAME'].unique():
                for name2 in prizePicks2['NAME'].unique():
                    if name1 == name2:
                        continue
                        
                    # Create unique key for this combination
                    combo_key = tuple(sorted([
                        (name1, cat1),
                        (name2, cat2)
                    ]))
                    
                    if combo_key in seen:
                        continue
                    seen.add(combo_key)

                    sim_result = bootstrapSim(
                        name1, name2, data, 
                        prizePicks1, prizePicks2,
                        simulations, stake, payout, weight,
                        stat_line1, stat_line2
                    )
                    
                    if sim_result is None:
                        continue

                    evs = {
                        'OVER/OVER': sim_result['OVER/OVER']['ev'],
                        'UNDER/UNDER': sim_result['UNDER/UNDER']['ev'],
                        'OVER/UNDER': sim_result['OVER/UNDER']['ev'],
                        'UNDER/OVER': sim_result['UNDER/OVER']['ev']
                    }
                    
                    probs = {
                        'OVER/OVER': sim_result['OVER/OVER']['prob'],
                        'UNDER/UNDER': sim_result['UNDER/UNDER']['prob'],
                        'OVER/UNDER': sim_result['OVER/UNDER']['prob'],
                        'UNDER/OVER': sim_result['UNDER/OVER']['prob']
                    }

                    best_type = max(evs, key=evs.get)
                    best_ev = evs[best_type]
                    best_prob = probs[best_type]

                    all_pairs.append({
                        'PLAYER 1': name1,
                        'CATEGORY 1': stat_line1,
                        'PLAYER 1 LINE': prizePicks1.loc[prizePicks1['NAME'] == name1, 'LINE'].values[0],
                        'PLAYER 2': name2,
                        'CATEGORY 2': stat_line2,
                        'PLAYER 2 LINE': prizePicks2.loc[prizePicks2['NAME'] == name2, 'LINE'].values[0],
                        'TYPE': best_type,
                        'EV': round(best_ev, 2),
                        'PROBABILITY': round(best_prob, 4),
                        'KELLY CRITERION': kelly_criterion(best_prob, payout, stake)
                    })

    return pd.DataFrame(all_pairs).sort_values('EV', ascending=False).reset_index(drop=True)

In [22]:
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    'player_blocks': 'BLK',
    'player_steals': 'STL',
    'player_turnovers': 'TOV'
}

# Run the analysis
results = prizePicksPairsEV(data, pp_data, propDict)

# Show top 10 highest EV combinations
results.head(10)

,PLAYER 1,CATEGORY 1,PLAYER 1 LINE,PLAYER 2,CATEGORY 2,PLAYER 2 LINE,TYPE,EV,PROBABILITY,KELLY CRITERION
0,Jalen Williams,PTS,0.5,Pascal Siakam,REB,7.5,OVER/UNDER,130.54,0.7685,0.7673
1,Jalen Williams,PTS,0.5,Luguentz Dort,REB,4.0,OVER/UNDER,120.75,0.7358,0.7345
2,Jalen Williams,PTS,0.5,Alex Caruso,REB,3.5,OVER/UNDER,113.86,0.7129,0.7115
3,Jalen Williams,PTS,0.5,Isaiah Hartenstein,PTS,6.5,OVER/OVER,104.18,0.6806,0.6790
4,Jalen Williams,PTS,0.5,T.J. McConnell,PTS,6.5,OVER/OVER,97.83,0.6594,0.6577
5,Jalen Williams,PTS,0.5,Shai Gilgeous-Alexander,STL,1.5,OVER/OVER,96.59,0.6553,0.6536
6,Jalen Williams,PTS,0.5,Ben Sheppard,AST,0.5,OVER/UNDER,96.35,0.6545,0.6528
7,Jalen Williams,PTS,0.5,Cason Wallace,PTS,6.0,OVER/UNDER,95.05,0.6502,0.6485
8,Jalen Williams,PTS,0.5,Obi Toppin,REB,4.0,OVER/UNDER,94.55,0.6485,0.6467
9,Jalen Williams,PTS,0.5,Obi Toppin,PTS,9.5,OVER/UNDER,91.15,0.6372,0.6354


In [8]:
results.to_csv('PROPS_EV/PrizePicksPairs(6_11_25).csv')

## 3 leg EVs for prizepicks and underdogs

In [13]:
def prizePicksTriosEV(data, prizePicks, propDict, simulations=10000, stake=100, payout=600, weight=0.8):
    def monte_carlo_sim_trio(name1, name2, name3, data, prizePicks1, prizePicks2, prizePicks3, simulations, stake, payout, weight, stat_line1, stat_line2, stat_line3):
        df1 = data.loc[data['PLAYER_NAME'] == name1]
        df2 = data.loc[data['PLAYER_NAME'] == name2]
        df3 = data.loc[data['PLAYER_NAME'] == name3]

        line1 = prizePicks1.loc[prizePicks1['NAME'] == name1, 'LINE'].values[0]
        line2 = prizePicks2.loc[prizePicks2['NAME'] == name2, 'LINE'].values[0]
        line3 = prizePicks3.loc[prizePicks3['NAME'] == name3, 'LINE'].values[0]

        mean1, std1 = df1[stat_line1].mean(), df1[stat_line1].std()
        mean2, std2 = df2[stat_line2].mean(), df2[stat_line2].std()
        mean3, std3 = df3[stat_line3].mean(), df3[stat_line3].std()
        
        if std1 == 0 or std2 == 0 or std3 == 0:
            return None

        sim1 = np.random.normal(mean1, std1, simulations)
        sim2 = np.random.normal(mean2, std2, simulations)
        sim3 = np.random.normal(mean3, std3, simulations)

        over1, under1 = sim1 > line1, sim1 <= line1
        over2, under2 = sim2 > line2, sim2 <= line2
        over3, under3 = sim3 > line3, sim3 <= line3

        mc_probs = {
            'OVER/OVER/OVER': np.mean(over1 & over2 & over3),
            'OVER/OVER/UNDER': np.mean(over1 & over2 & under3),
            'OVER/UNDER/OVER': np.mean(over1 & under2 & over3),
            'OVER/UNDER/UNDER': np.mean(over1 & under2 & under3),
            'UNDER/OVER/OVER': np.mean(under1 & over2 & over3),
            'UNDER/OVER/UNDER': np.mean(under1 & over2 & under3),
            'UNDER/UNDER/OVER': np.mean(under1 & under2 & over3),
            'UNDER/UNDER/UNDER': np.mean(under1 & under2 & under3),
        }

        z_over1 = zscore_prob(mean1, std1, line1, side='over')
        z_under1 = 1 - z_over1
        z_over2 = zscore_prob(mean2, std2, line2, side='over')
        z_under2 = 1 - z_over2
        z_over3 = zscore_prob(mean3, std3, line3, side='over')
        z_under3 = 1 - z_over3

        z_probs = {
            'OVER/OVER/OVER': z_over1 * z_over2 * z_over3,
            'OVER/OVER/UNDER': z_over1 * z_over2 * z_under3,
            'OVER/UNDER/OVER': z_over1 * z_under2 * z_over3,
            'OVER/UNDER/UNDER': z_over1 * z_under2 * z_under3,
            'UNDER/OVER/OVER': z_under1 * z_over2 * z_over3,
            'UNDER/OVER/UNDER': z_under1 * z_over2 * z_under3,
            'UNDER/UNDER/OVER': z_under1 * z_under2 * z_over3,
            'UNDER/UNDER/UNDER': z_under1 * z_under2 * z_under3,
        }

        probs = {
            k: weight * mc_probs[k] + (1 - weight) * z_probs[k]
            for k in mc_probs
        }

        evs = {k: round((probs[k] * payout) - stake, 2) for k in probs}

        return {
            combo: {'prob': round(probs[combo], 4), 'ev': evs[combo]}
            for combo in probs
        }

    all_trios = []
    seen = set()

    # Iterate through all categories for player 1
    for cat1 in propDict:
        prizePicks1 = prizePicks[prizePicks['CATEGORY'] == cat1]
        stat_line1 = propDict[cat1]
        
        # Compare against all categories for player 2
        for cat2 in propDict:
            prizePicks2 = prizePicks[prizePicks['CATEGORY'] == cat2]
            stat_line2 = propDict[cat2]
            
            # Compare against all categories for player 3
            for cat3 in propDict:
                prizePicks3 = prizePicks[prizePicks['CATEGORY'] == cat3]
                stat_line3 = propDict[cat3]
                
                for name1 in prizePicks1['NAME'].unique():
                    for name2 in prizePicks2['NAME'].unique():
                        for name3 in prizePicks3['NAME'].unique():
                            # Skip if any players are the same
                            if name1 == name2 or name1 == name3 or name2 == name3:
                                continue
                                
                            # Create unique key for this combination
                            combo_key = tuple(sorted([
                                (name1, cat1),
                                (name2, cat2),
                                (name3, cat3)
                            ]))
                            
                            if combo_key in seen:
                                continue
                            seen.add(combo_key)

                            sim_result = monte_carlo_sim_trio(
                                name1, name2, name3, data, 
                                prizePicks1, prizePicks2, prizePicks3,
                                simulations, stake, payout, weight,
                                stat_line1, stat_line2, stat_line3
                            )
                            
                            if sim_result is None:
                                continue

                            evs = {combo: sim_result[combo]['ev'] for combo in sim_result}
                            probs = {combo: sim_result[combo]['prob'] for combo in sim_result}

                            best_type = max(evs, key=evs.get)
                            best_ev = evs[best_type]
                            best_prob = probs[best_type]

                            all_trios.append({
                                'PLAYER 1': name1,
                                'CATEGORY 1': stat_line1,
                                'PLAYER 1 LINE': prizePicks1.loc[prizePicks1['NAME'] == name1, 'LINE'].values[0],
                                'PLAYER 2': name2,
                                'CATEGORY 2': stat_line2,
                                'PLAYER 2 LINE': prizePicks2.loc[prizePicks2['NAME'] == name2, 'LINE'].values[0],
                                'PLAYER 3': name3,
                                'CATEGORY 3': stat_line3,
                                'PLAYER 3 LINE': prizePicks3.loc[prizePicks3['NAME'] == name3, 'LINE'].values[0],
                                'TYPE': best_type,
                                'EV': round(best_ev, 2),
                                'PROBABILITY': round(best_prob, 4),
                                'KELLY CRITERION': kelly_criterion(best_prob, payout, stake)
                            })

    return pd.DataFrame(all_trios).sort_values('EV', ascending=False).reset_index(drop=True)

In [15]:
# Example usage
propDict = {
    'player_points': 'PTS',
    'player_rebounds': 'REB',
    'player_assists': 'AST',
    'player_blocks': 'BLK',
    'player_steals': 'STL',
    'player_turnovers': 'TOV'
}

# Run the 3-pick analysis (note the higher payout of 600 for 3-picks vs 300 for 2-picks)
trio_results = prizePicksTriosEV(data, pp_data, propDict, payout=600)

# Show top 10 highest EV combinations
trio_results.head(10)

# Save results
# trio_results.to_csv('PROPS_EV/PrizePicksTrios(6_11_25).csv')

,PLAYER 1,CATEGORY 1,PLAYER 1 LINE,PLAYER 2,CATEGORY 2,PLAYER 2 LINE,PLAYER 3,CATEGORY 3,PLAYER 3 LINE,TYPE,EV,PROBABILITY,KELLY CRITERION
0,Jalen Williams,PTS,0.5,Isaiah Hartenstein,PTS,6.5,Pascal Siakam,REB,7.5,OVER/OVER/UNDER,229.64,0.5494,0.5485
1,Jalen Williams,PTS,0.5,Pascal Siakam,REB,7.5,Alex Caruso,REB,3.5,OVER/UNDER/UNDER,208.75,0.5146,0.5136
2,Jalen Williams,PTS,0.5,Isaiah Hartenstein,PTS,6.5,Alex Caruso,REB,3.5,OVER/OVER/UNDER,201.03,0.5017,0.5007
3,Jalen Williams,PTS,0.5,T.J. McConnell,PTS,6.5,Pascal Siakam,REB,7.5,OVER/OVER/UNDER,199.28,0.4988,0.4978
4,Jalen Williams,PTS,0.5,Pascal Siakam,REB,7.5,Kenrich Williams,REB,1.5,OVER/UNDER/OVER,198.73,0.4979,0.4969
5,Jalen Williams,PTS,0.5,Shai Gilgeous-Alexander,PTS,33.5,Pascal Siakam,REB,7.5,OVER/UNDER/UNDER,193.04,0.4884,0.4874
6,Jalen Williams,PTS,0.5,T.J. McConnell,PTS,6.5,Isaiah Hartenstein,PTS,6.5,OVER/OVER/OVER,192.17,0.4869,0.4859
7,Jalen Williams,PTS,0.5,Pascal Siakam,REB,7.5,Isaiah Hartenstein,REB,7.0,OVER/UNDER/OVER,191.07,0.4851,0.4841
8,Jalen Williams,PTS,0.5,Pascal Siakam,REB,7.5,Tyrese Haliburton,AST,8.0,OVER/UNDER/OVER,188.52,0.4809,0.4799
9,Jalen Williams,PTS,0.5,Isaiah Hartenstein,PTS,6.5,Kenrich Williams,REB,1.5,OVER/OVER/OVER,185.77,0.4763,0.4753


⚙️ Hybrid Approach Recommended
For NBA prop betting, a hybrid workflow often works best:
- Train an ML model to predict the expected prop value using features like usage rate, matchup context, rest, etc.
- Simulate multiple game scenarios via Monte Carlo, using your ML model’s predictive distribution (e.g., add residual noise or use a Bayesian framework).
- Bootstrap on your historical residuals to calibrate confidence intervals around those predictions.

This combines:
ML’s ability to capture complex relationships,
Monte Carlo’s capacity for modeling variability and outputting a full prob. distribution,
Bootstrap’s non-parametric estimate of uncertainty—all necessary to assess value and risk correctly 

🗓️ Practical Takeaway
Need a full probability distribution for value + risk? MC sim driven by your ML model is essential.


📌 Final Recommendation
Use Machine Learning to predict the mean prop, then Monte Carlo to simulate possible outcomes (using those ML predictions with added variance), and Bootstrap to quantify your confidence in those outcomes. This hybrid setup offers the most accurate, actionable probabilities for NBA prop betting.
